# Introduction
After the previous commits in notebook 33, Ben noticed that there are some issues that arise that prevent the model from working in MatLab. Here I will fix those issues.

In [ ]:
import cameo
import pandas as pd
import cobra.io

In [ ]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

## Modify EC code in annotations
Here, the multiple EC codes are stored in the annotations as strings instead of as lists. This is a problem and will not be tolerated by CobraToolbox in Matlab and so needs to be fixed.

I also observed one metabolite still has 'EC ' in the annotation, I will write a script to remove it from whereever it is still present.

In [113]:
for rct in model.reactions:
    try:
        anno= rct.annotation['ec-code']
        if type(anno) == str:
            if len(anno) > 7:
                anno = anno.split(sep = ' ')
                rct.annotation['ec-code'] = anno
            else:
                continue
        else: 
            continue
    except KeyError: #rcts without ec-code
        continue

In [108]:
#remove any 'EC' in the annotation field
for rct in model.reactions:
    try:
        anno = rct.annotation['ec-code']
        for a in anno:
            if a in 'EC':
                anno.remove(a)
            else:
                continue
    except KeyError: #where there is no ec-code
        continue

AttributeError: 'str' object has no attribute 'remove'

In [106]:
anno

'EC 1.3.1.14'

Also, I see that there are some semicolons in the notes that were lifted to the annotation. I will remove them from the annotation field.

In [ ]:
for rct in model.reactions:
    try:
        anno = rct.annotation['ec-code']

In [ ]:
rct = model.reactions.BUTACPAT

In [ ]:
anno = rct.annotation['ec-code']

In [ ]:
anno

In [92]:
for ec in anno:
    ec.replace(';','')

In [93]:
ec

'2.3.1.179'

In [94]:
anno

['2.3.1.-;', '2.3.1.41', '2.3.1.85', '2.3.1.86', '2.3.1.179']

In [ ]:
anno[0].replace(';','')

In [ ]:
#spotted problem:
SUCHY has duplicate: one with ;
    same for BUTACPAT (maybe just write code to get rid of it in this annotation field)
    HEXACPAT
    meta_R_OCTACPAT

In [114]:
#Save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

In [32]:
rct = model.reactions.ACACT1

In [33]:
rct.annotation

{'sbo': 'SBO:0000176',
 'ec-code': '2.3.1.9',
 'kegg.reaction': 'R00238',
 'metanetx.reaction': 'MNXR95194',
 'rhea': '21036'}

In [27]:
anno = rct.annotation['ec-code']

In [28]:
type(anno)

list

In [19]:
anno = anno.split(sep = ' ')

In [20]:
rct.annotation['ec-code'] = anno

In [21]:
rct.annotation

{'sbo': 'SBO:0000176',
 'ec-code': ['1.7.2.1', '1.9.3.1'],
 'kegg.reaction': 'R00082',
 'metanetx.reaction': 'MNXR106381'}

In [ ]:
check this also for kegg.reactions
